In [1]:
import pandas as pd
import nltk
from tqdm.notebook import tqdm
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from operator import itemgetter

In [143]:
# Кластеры для заголовков
df_publ = pd.read_excel('/Users/thebest/Desktop/python/Учеба/курсовая работа /sciencedir_title — копия.xlsx')
texts = list(df_publ['Заголовок'])[:-200]
texts_test = list(df_publ['Заголовок'])[-200:]

tokkens_lists = []
for sentence in (texts):
    tokkens_lists.append(nltk.word_tokenize(sentence))
    
morph = pymorphy2.MorphAnalyzer()
teg_list = ['NOUN','ADJF','VERB','INFN']
tokkens_lem_lists = []
for tokkens_list in (tokkens_lists):
    tokkens_lem_list = []
    for token in tokkens_list:
        token_lem = morph.parse(token)
        if token_lem[0].tag.POS in teg_list:
            tokkens_lem_list.append(token_lem[0].normal_form)
    tokkens_lem_lists.append(tokkens_lem_list)   
    
texts_lem_lists = []
for sentece_lem_list in (tokkens_lem_lists):
    texts_lem_lists.append(' '.join(sentece_lem_list))    
    
tfidfconverter = TfidfVectorizer(max_features = 1500)
X = tfidfconverter.fit_transform(texts_lem_lists).toarray()

lda = LatentDirichletAllocation(n_components = clusters_num,
                                learning_method = "batch",
                                max_iter = 25,
                                random_state = 0)
document_topics = lda.fit_transform(X)

sorting = np.argsort(lda.components_, axis = 1)[:, ::-1]
feature_names = np.array(list(dict(sorted(tfidfconverter.vocabulary_.items(), key = itemgetter(1))).keys()))   
clusters_tokens = []
for num in (range (0, clusters_num)):
    tokens_list = list(feature_names[list(sorting[num])])
    clusters_tokens.append(tokens_list)   
df_clust_title = pd.DataFrame(clusters_tokens).T
df_clust_title.to_excel('/Users/thebest/Desktop/python/Учеба/курсовая работа /результат/Кластеры_заголовки_new.xlsx')

In [140]:
df_clust_title

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,подход,сетевой,стрит,криптовалютный,детерминант,биткоина,биткоина,криптовалюта,перспектива,обучение,фьючерсный,африканский,биткоина,биткоина,биткойн,биткоина,равновесный,криптовалюта,цена,равновесие
1,потенциал,анализ,любимица,глобальный,ликвидность,год,цена,являться,источник,глубокий,спотовой,фондовый,активность,криптовалюта,спрос,криптовалюта,модель,контекст,блокчейн,биткоин
2,динамический,возможность,настроение,многомасштабный,биржа,конец,стоимость,спекулятивный,статистический,открытие,рынок,рынок,покупать,рынок,эластичность,рынок,ценообразование,фиатный,транзакция,криптовалюта
3,биткойне,арбитраж,уолл,характеристика,алгоритм,введение,торговля,владение,объяснение,валюта,эффективность,криптовалюта,культурный,криптовалютный,исследование,криптовалютный,общий,функция,перспектива,генератор
4,использование,высокочастотный,регулирование,рынок,сравнение,обрушить,криптовалютный,горизонт,арбитражный,фиатной,ценообразование,многомерный,детерминант,цена,арбитраж,цена,бум,деньга,блокировочный,афера
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,обрушить,предиктор,обрушить,предиктор,привлекательность,настроение,расхождение,потенциал,предиктор,привлекательность,различный,превращение,открытие,разный,потенциал,разный,потенциал,предиктор,реагировать,превращение
178,обучение,привлекательность,обучение,привлекательность,равновесный,обучение,реагировать,превращение,привлекательность,равновесный,разный,предиктор,децентрализованный,различный,портфель,различный,предиктор,привлекательность,регулирование,потенциал
179,открытие,равновесный,открытие,равновесный,различный,открытие,реальный,предиктор,равновесный,различный,расхождение,привлекательность,сетевой,равновесный,покупка,равновесный,привлекательность,равновесный,риск,портфель
180,переоценка,различный,переоценка,разный,разный,переоценка,регулирование,привлекательность,различный,разный,реагировать,равновесный,год,равновесие,покупать,равновесие,различный,различный,рост,покупать


In [142]:

# 13, 18, 14  - близки к самой торговой стратегии 
# 15,10 - прогнозирование рынков
clusters_norm_nums = [10,13,14,15,18]

clusters_solution = []
for marks_list in tqdm(document_topics):
    clusters_solution.append(marks_list[clusters_norm_nums])
y = []
for marks_list in tqdm(clusters_solution):
    m = list(marks_list)
    y.append(m.index(max(m)))
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

classifier = RandomForestClassifier(n_estimators = 100, random_state = 0 )
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))

scores = classifier.predict_proba(tfidfconverter.transform(texts_test).toarray())
df_lists = []
index = 0
for t in texts_test:
    df_lists.append(list(np.append(t,scores[index])))
    index += 1
pd.DataFrame(df_lists)
pd.DataFrame(df_lists).to_excel('/Users/thebest/Desktop/python/Учеба/курсовая работа /результат/Осмысление_кластеры_заголовки_new.xlsx')

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

0.6


In [144]:
# Кластеры для издательств

df_publ = pd.read_excel('/Users/thebest/Desktop/python/Учеба/курсовая работа /sciencedir_title — копия.xlsx')
texts = list(df_publ['Издательство'])[:-200]
texts_test = list(df_publ['Издательство'])[-200:]

tokkens_lists = []
for sentence in (texts):
    tokkens_lists.append(nltk.word_tokenize(sentence))
    
morph = pymorphy2.MorphAnalyzer()
teg_list = ['NOUN','ADJF','VERB','INFN']
tokkens_lem_lists = []
for tokkens_list in (tokkens_lists):
    tokkens_lem_list = []
    for token in tokkens_list:
        token_lem = morph.parse(token)
        if token_lem[0].tag.POS in teg_list:
            tokkens_lem_list.append(token_lem[0].normal_form)
    tokkens_lem_lists.append(tokkens_lem_list)   
    
texts_lem_lists = []
for sentece_lem_list in (tokkens_lem_lists):
    texts_lem_lists.append(' '.join(sentece_lem_list))    
    
tfidfconverter = TfidfVectorizer(max_features = 1500)
X = tfidfconverter.fit_transform(texts_lem_lists).toarray()

lda = LatentDirichletAllocation(n_components = clusters_num,
                                learning_method = "batch",
                                max_iter = 25,
                                random_state = 0)
document_topics = lda.fit_transform(X)

sorting = np.argsort(lda.components_, axis = 1)[:, ::-1]
feature_names = np.array(list(dict(sorted(tfidfconverter.vocabulary_.items(), key = itemgetter(1))).keys()))   
clusters_tokens = []
for num in (range (0, clusters_num)):
    tokens_list = list(feature_names[list(sorting[num])])
    clusters_tokens.append(tokens_list)   
df_clust_publ = pd.DataFrame(clusters_tokens).T
df_clust_publ.to_excel('/Users/thebest/Desktop/python/Учеба/курсовая работа /результат/Кластеры_издательства_new.xlsx')

In [145]:
df_clust_publ

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,анализ,математический,применение,применение,применение,отчёт,применение,исследование,статистический,применение,изменение,экономический,финансы,информатика,эмпирический,применение,применение,экономика,журнал,ежеквартальный
1,обзор,общественный,финансы,финансы,финансы,физика,финансы,письмо,механика,финансы,технологический,письмо,область,телематика,журнал,финансы,финансы,североамериканский,рынок,экономика
2,международный,наука,финансовый,финансовый,финансовый,применение,финансовый,финансовый,приложение,финансовый,социальный,применение,бизнес,применение,финансы,финансовый,финансовый,журнал,финансовый,обзор
3,финансовый,поддержка,международный,международный,международный,финансы,международный,применение,применение,международный,прогнозирование,финансы,исследовательский,финансы,применение,международный,международный,финансы,деньга,финансы
4,применение,принятие,журнал,журнал,журнал,финансовый,журнал,международный,финансы,журнал,применение,финансовый,исследование,финансовый,финансовый,журнал,журнал,финансовый,институт,применение
5,исследование,решение,письмо,письмо,письмо,международный,письмо,финансы,финансовый,письмо,международный,журнал,письмо,международный,международный,письмо,письмо,применение,международный,письмо
6,финансы,система,исследование,исследование,исследование,письмо,исследование,рынок,международный,исследование,финансы,международный,международный,исследование,исследование,исследование,исследование,письмо,финансы,финансовый
7,наука,применение,обзор,обзор,обзор,исследование,обзор,исследовательский,письмо,обзор,финансовый,деньга,деньга,письмо,письмо,обзор,обзор,международный,экономика,журнал
8,механика,международный,деньга,деньга,деньга,журнал,деньга,обзор,деньга,деньга,рынок,институт,журнал,журнал,обзор,деньга,деньга,исследование,применение,международный
9,математический,финансовый,институт,институт,институт,институт,институт,журнал,журнал,институт,исследовательский,исследование,финансовый,институт,общественный,институт,институт,обзор,исследовательский,исследование


In [147]:

# 4,5 - инвест журналы
# 7,9 - экномический анализ
# 10 - машинное обучение и ИИ в области инвестиций
clusters_norm_nums = [4, 5, 7, 9, 10]

clusters_solution = []
for marks_list in (document_topics):
    clusters_solution.append(marks_list[clusters_norm_nums])
y = []
for marks_list in (clusters_solution):
    m = list(marks_list)
    y.append(m.index(max(m)))
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

classifier = RandomForestClassifier(n_estimators = 1000, random_state = 0 )
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))

scores = classifier.predict_proba(tfidfconverter.transform(texts_test).toarray())
df_lists = []
index = 0
for t in texts_test:
    df_lists.append(list(np.append(t,scores[index])))
    index += 1
pd.DataFrame(df_lists)
pd.DataFrame(df_lists).to_excel('/Users/thebest/Desktop/python/Учеба/курсовая работа /результат/Осмысление_кластеры_издательства_new.xlsx')

1.0
